![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/features.txt"
filename_labels = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "./human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt"

In [2]:
features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])
subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])

X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())

y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [3]:
%%time
clf = DecisionTreeClassifier(ccp_alpha= 0.01)
clf.fit(X_train, y_train)

predict = clf.predict(X_test)
train_score = clf.score(X_train, y_train)
valid_score = accuracy_score(y_test, predict)

print(f'Acurácia da árvore na base de treino:    {train_score:.3f}')
print(f'Acurácia da árvore na base de teste:     {valid_score:.3f}')

Acurácia da árvore na base de treino:    0.909
Acurácia da árvore na base de teste:     0.838
CPU times: total: 5.92 s
Wall time: 10.6 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [4]:
%%time
prcomp = PCA(n_components=1).fit(X_train)
pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

clf_2 = DecisionTreeClassifier(ccp_alpha=0.01).fit(pc_treino,y_train)
predict_2 = clf_2.predict(pc_teste)
accuracy_score(y_test, predict_2)

train_score = clf_2.score(pc_treino, y_train)
valid_score = accuracy_score(y_test, predict_2)

print(f'Acurácia da árvore na base de treino:    {train_score:.3f}')
print(f'Acurácia da árvore na base de teste:     {valid_score:.3f}')

Acurácia da árvore na base de treino:    0.462
Acurácia da árvore na base de teste:     0.431
CPU times: total: 93.8 ms
Wall time: 174 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [5]:
%%time
components = [1, 2, 5, 10, 50]
acuracias_treino = []
acuracias_teste = []

for n in components:
    prcomp = PCA(n_components=n).fit(X_train)
    pc_treino = prcomp.transform(X_train)
    pc_teste  = prcomp.transform(X_test)
    
    clf = DecisionTreeClassifier(ccp_alpha=0.01).fit(pc_treino,y_train)
    predict = clf.predict(pc_teste)
    acuracias_teste.append(accuracy_score(y_test, predict))
    acuracias_treino.append(clf.score(pc_treino, y_train))

CPU times: total: 984 ms
Wall time: 2.17 s


In [6]:
print(f'Acurácia da árvore na base de treino com 1 componente:    {acuracias_teste[0]:.3f}')
print(f'Acurácia da árvore na base de treino com 2 componentes:    {acuracias_teste[1]:.3f}')
print(f'Acurácia da árvore na base de treino com 5 componentes:    {acuracias_teste[2]:.3f}')
print(f'Acurácia da árvore na base de treino com 10 componentes:    {acuracias_teste[3]:.3f}')
print(f'Acurácia da árvore na base de treino com 50 componentes:    {acuracias_teste[4]:.3f}')

print('\n################################################################\n')

print(f'Acurácia da árvore na base de teste com 1 componente:     {acuracias_treino[0]:.3f}')
print(f'Acurácia da árvore na base de teste com 2 componente:     {acuracias_treino[1]:.3f}')
print(f'Acurácia da árvore na base de teste com 5 componente:     {acuracias_treino[2]:.3f}')
print(f'Acurácia da árvore na base de teste com 10 componente:     {acuracias_treino[3]:.3f}')
print(f'Acurácia da árvore na base de teste com 50 componente:     {acuracias_treino[4]:.3f}')

Acurácia da árvore na base de treino com 1 componente:    0.431
Acurácia da árvore na base de treino com 2 componentes:    0.538
Acurácia da árvore na base de treino com 5 componentes:    0.725
Acurácia da árvore na base de treino com 10 componentes:    0.801
Acurácia da árvore na base de treino com 50 componentes:    0.815

################################################################

Acurácia da árvore na base de teste com 1 componente:     0.462
Acurácia da árvore na base de teste com 2 componente:     0.537
Acurácia da árvore na base de teste com 5 componente:     0.767
Acurácia da árvore na base de teste com 10 componente:     0.826
Acurácia da árvore na base de teste com 50 componente:     0.826


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

A principal diferença foi o tempo de processamento, sendo que este caiu em mais de 10, sendo que até quando foram testadas PCAs com diferente números<br> de componentes o tempo de processamento foi menor que o tempo de uma árvore com os dados brutos.<br> A acurácia porém, caiu consideravelmente com PCAs de apenas um componente (principalmente nas bases de treino), porém, PCAs de 50 componentes<br> obtiveram acurácias altamente semelhantes aos dados brutos nas bases de teste (0,838 e 0,826), com uma fração do tempo necessário.